In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms,models
from collections import OrderedDict
from tqdm import tqdm

In [2]:
data_dir = 'Cat_Dog_data'

train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])

])

trainset = datasets.ImageFolder(data_dir+'/train/', transform = train_transform)
testset = datasets.ImageFolder(data_dir+'/test/', transform = test_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True) 
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)

In [3]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
for params in model.parameters():
    params.requires_grad=False
    
classifier = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(512,128)),
    ('relu',nn.ReLU()),
    ('fc2',nn.Linear(128,2)),
    ('output',nn.LogSoftmax(dim=1))
]))

model.fc = classifier

In [5]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [ ]:
epochs = 1
steps = 0
print_every = 3
train_loss = 0
train_accuracy=0
for e in range(epochs):

    for images,labels in tqdm(trainloader):
        steps +=1
        outputs = model(images)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        preds = torch.exp(outputs)
        top_class= torch.argmax(preds,dim=1)
        equals = top_class == labels.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor))
        
    
        if(steps % print_every == 0) :
            test_loss=0
            test_accuracy=0
            with torch.no_grad():
                model.eval()
                for images,labels in tqdm(testloader):
                        predictions = model(images)
                        loss = criterion(predictions,labels)
                        test_loss += loss.item()
                        predictions = torch.exp(predictions)
                        top_class = torch.argmax(predictions,dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        test_accuracy += torch.mean(equals.type(torch.FloatTensor))

            print("Epoch: {}/{}.. ".format(e+1, epochs),
                "Training Loss: {:.3f}.. ".format(train_loss/print_every),
                "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                "Train Accuracy: {:.3f}".format(train_accuracy/print_every),
                "Test Accuracy: {:.3f}".format(test_accuracy/len(testloader)))

            train_loss = 0
            train_accuracy=0
            model.train()

 81%|██████████████████████████████████████████████████████████████████▍               | 64/79 [06:45<01:32,  6.18s/it]


  0%|▎                                                                             | 3/704 [08:36<30:19:44, 155.76s/it]

Epoch: 1/1..  Training Loss: 0.642..  Test Loss: 0.460..  Train Accuracy: 0.635 Test Accuracy: 0.817


 81%|██████████████████████████████████████████████████████████████████▍               | 64/79 [06:22<01:28,  5.91s/it]


  1%|▋                                                                             | 6/704 [16:43<38:18:48, 197.61s/it]

...

Epoch: 1/1..  Training Loss: 0.257..  Test Loss: 0.191..  Train Accuracy: 0.885 Test Accuracy: 0.917


 81%|██████████████████████████████████████████████████████████████████▍               | 64/79 [08:05<01:47,  7.15s/it]


  6%|████▊                                                                      | 45/704 [2:33:53<54:27:42, 297.52s/it]

Epoch: 1/1..  Training Loss: 0.244..  Test Loss: 0.266..  Train Accuracy: 0.875 Test Accuracy: 0.886


  0%|                                                                                           | 0/79 [00:00<?, ?it/s]